In [9]:
import json
import pdfplumber

questions = json.load(open("questions.json"))
print(questions[0])

pdf = pdfplumber.open("初赛训练数据集.pdf")
len(pdf.pages) # 页数
pdf.pages[0].extract_text() # 读取第一页内容

{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '', 'reference': ''}


'欢迎\n感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。\n首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。\n本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际\n接收的车辆为准。\n如您有任何问题，或需要预约服务，请拨打电话4006-010101联系我们。您也可以开车前往Lynk&Co领克中心。\n在抵达之前，请您注意驾车安全。\n©领克汽车销售有限公司'

In [23]:
pdfplumber

<module 'pdfplumber' from '/usr/local/anaconda3/envs/rag/lib/python3.9/site-packages/pdfplumber/__init__.py'>

In [24]:
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })

In [25]:
pdf_content[12]

{'page': 'page_13',
 'content': '前言\n原厂精装附件、选装装备和改装 在使用无线电设备时应当严格遵守以下规定：\n■ 切勿自行安装或变更无线电设备，否则可能会造成某些电气系统\nLynk&Co领克原厂精装附件都是经过测试的，确保能够与您车辆的性 故障。\n能、安全和排放系统很好地兼容。为了尽可能的保证车辆性能与安\n■ 使用无线电设备时，请遵循车辆所在地法规。\n全，建议您优先使用Lynk&Co领克原厂精装附件。\n■ 切勿将车辆停放在变压器等电磁干扰较强的地方。\n如果您安装非Lynk&Co领克原厂精装附件，可能会对您车辆的性能和\n安全系统产生不利影响。因此产生的任何问题，Lynk&Co领克将不会\n警告！\n承担责任。\n■ 使用植入式心脏起搏器、心脏再同步治疗起搏器或植入式心脏复\n您可以订购某些经过Lynk&Co领克认证的选装装备。由于销售市场不 律除颤器的人士应与无线电设备保持适当的距离。无线电波可能\n同，专门为某些国家或地区提供的选装装备可能不适用于您的地区。 影响此类设备的工作。如有必要，请禁用无线电设备。\n在您订购选装装备之前，请查询当地法律法规，并联系Lynk&Co领克 ■ 请勿将无线电设备安装在安全气囊展开的范围内。\n中心。如果您使用了未经Lynk&Co领克认证或不适用您所在地区的选\n■ 对于使用其他电子医疗设备的人士，应向该设备制造商咨询无线\n装装备，引起的任何问题，Lynk&Co领克将不会承担责任。\n电波对设备工作的影响情况。无线电波可能会对这类医疗设备的\n工作产生难以预料的影响。\n切勿私自改装车辆，否则将会影响车辆的操控性、安全性和耐久性，\n同时有可能会触犯当地政府条例。因私自改装引起的车辆损坏或性能\n问题，Lynk&Co领克将不承担任何责任。 注意！\n■ 自行安装或变更无线电设备所造成的影响和损失，Lynk&Co领克\n警告！ 将不承担任何责任。\n■ 无线电设备的电波，会对安全气囊的控制装置产生影响。\n■ Lynk&Co领克只对经Lynk&Co领克测试且认证的原厂精装附件\n和选装装备负责。因安装非原厂精装附件或非选装装备而导致的\n人身伤亡、车损或费用，Lynk&Co领克不提供任何保证且不承担\n任何责任，不履行任何义务。\n所有权变更\n如果您需要变更车辆所有权，请携带本人

In [26]:
import jieba
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

# 对提问和PDF内容进行分词
question_words = [' '.join(jieba.lcut(x['question'])) for x in questions]
pdf_content_words = [' '.join(jieba.lcut(x['content'])) for x in pdf_content]

tfidf = TfidfVectorizer()
tfidf.fit(question_words + pdf_content_words)

# 提取TFIDF
question_feat = tfidf.transform(question_words)
pdf_content_feat = tfidf.transform(pdf_content_words)

# 进行归一化
question_feat = normalize(question_feat)
pdf_content_feat = normalize(pdf_content_feat)

# 检索进行排序
for query_idx, feat in enumerate(question_feat):
    score = feat @ pdf_content_feat.T
    score = score.toarray()[0]
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)

# 生成提交结果
# https://competition.coggle.club/
# with open('submit.json', 'w', encoding='utf8') as up:
#     json.dump(questions, up, ensure_ascii=False, indent=4)

In [27]:
questions[10]

{'question': '装载货物时，车内有哪些储物空间可以利用？', 'answer': '', 'reference': 'page_19'}

In [28]:
question_words[7]

'问题 ： 在 国家 环保法 要求 下 ， 哪些 情况 下 需要 对 车辆 进行 报废 处理 ？'

In [16]:
pdf_content_words[6]

'目录 \n 车辆 标识 ............................................................... 347 \n 车辆 参数 ............................................................... 349 \n 缩略语 和 术语 .......................................................... 352 \n 7'

In [29]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./m3e-base")

In [30]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [31]:
question_sentences = [x['question'] for x in questions]
pdf_content_sentences = [x['content'] for x in pdf_content]

question_embeddings = model.encode(question_sentences, normalize_embeddings=True)
pdf_embeddings = model.encode(pdf_content_sentences, normalize_embeddings=True)

for query_idx, feat in enumerate(question_embeddings):
    score = feat @ pdf_embeddings.T
    max_score_page_idx = score.argsort()[-1] + 1
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx)


RuntimeError: MPS backend out of memory (MPS allocated: 2.58 GB, other allocations: 3.99 GB, max allowed: 6.77 GB). Tried to allocate 214.88 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
questions[13]